<a href="https://colab.research.google.com/github/whoami-Lory271/DL-project/blob/main/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pytorch-lightning --quiet
!pip install torchmetrics --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 825.8/825.8 KB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 517.2/517.2 KB 3.6 MB/s eta 0:00:00


In [2]:
!pip install -U --no-cache-dir gdown --pre

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: gdown
    Found existing installation: gdown 4.4.0
    Uninstalling gdown-4.4.0:
      Successfully uninstalled gdown-4.4.0


In [13]:
from numpy import array
from sklearn.preprocessing import OneHotEncoder
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.modules.normalization import LayerNorm
import pickle
import os
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToTensor
from pathlib import Path
import pandas as pd
import numpy as np
import csv
import torchmetrics
import pytorch_lightning as pl
from pytorch_lightning.callbacks.progress import TQDMProgressBar
import gdown
import random

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Helper Function

In [5]:
def create_padding(encode, vocab, max_len):
    enc = np.array(encode)
    encoded = np.zeros(max_len + 2)
    for k in range(len(enc)):
      encoded[k] = enc[k]
    return encoded

In [6]:
def get_padding_mask(X,inverse=False):
  pad = None
  if inverse == False:
    pad = X == 0
  else:
    pad = X != 0
  padding_mask = pad.repeat(1,1,X.shape[1]).reshape((X.shape[0],X.shape[1],X.shape[1]))
  if inverse == False:
    padding_mask[pad] = True
  return padding_mask

##Dataset

In [7]:

url = "https://drive.google.com/drive/folders/1gt_OmBTq_T-0EZbAIKDq6pfXRvuHCmq7?usp=sharing" #algebra
gdown.download_folder(url = url, quiet = True, use_cookies=False)  

url = "https://drive.google.com/drive/folders/1h-RQsLMDcw3yVZGMtsInx-0ePPYW-C_M?usp=sharing" #calculus
gdown.download_folder(url = url, quiet = True, use_cookies=False)

url = "https://drive.google.com/drive/folders/1gu-yWS9gtcLPXjvpBkQDPmgRcCpcpr2t?usp=sharing" #probability
gdown.download_folder(url = url, quiet = True, use_cookies=False)

url = "https://drive.google.com/drive/folders/1dK0ifABFPmXYwsJQow5038yXuUA0Pqy_?usp=sharing" #interpolate
gdown.download_folder(url = url, quiet = True, use_cookies=False)

!gdown "1-7ies2HEUy56W8akeHlBN-XkPDPy9uh0" #vocab


Downloading...
From: https://drive.google.com/uc?id=1-7ies2HEUy56W8akeHlBN-XkPDPy9uh0
To: /content/vocabs
100% 376/376 [00:00<00:00, 811kB/s]


In [297]:
class MathematicsDataset(Dataset):
  def __init__(self, module, train_type, max_len_quest=160, max_len_answ=30):
    self.max_len_quest = max_len_quest
    self.max_len_answ = max_len_answ
    #unpickle the vocabulary
    with open(os.path.join("/content",'vocabs'),'rb') as infile:
      self.vocab = pickle.load(infile)

    modules = ["algebra","calculus", "probability","interpolate"]
    assert module in modules, f"the module should be algebra , calculus, probability or interpolate!"
    if module in ["algebra","calculus", "probability"]:
      train_types =  ["easy","medium", "hard", "complete"]
      assert train_type in train_types, f"the train_type for {module} should be easy, medium or hard!"
    elif module in ["interpolate"]:
      train_types = ["algebra","calculus", "probability"]
      assert train_type in train_types, f"the train_type for {module} should be algebra, calculus or proability!"

    self.path = "/content/" + module + "_pickle" +"/" + train_type

    #unpickle the qestions
    with open(os.path.join(self.path,"questions"),'rb') as infile:
      self.quest = pickle.load(infile)

    #unpickle the answers
    with open(os.path.join(self.path,"answers"),'rb') as infile:
      self.answ = pickle.load(infile)
  
  def __len__(self):
    return len(self.quest)

  def __getitem__(self, idx):
    assert(idx  < len(self.quest)) #indices should start from 0 to len - 1 (there are 666666 elements or 1999998 elements)
    question = self.quest[idx]["Encoding"]
    encoding1 = torch.from_numpy(create_padding(question, self.vocab, self.max_len_quest))
    encoding1 = encoding1.type(torch.int64)
      
    answer = self.answ[idx]["Encoding"]
    encoding2 = torch.from_numpy(create_padding(answer, self.vocab, self.max_len_answ))
    encoding2 = encoding2.type(torch.int64)
      
    return encoding1, encoding2 

In [298]:
class PlMathematicsDataset(pl.LightningDataModule):
    def __init__(self, module, train_type, max_len_quest=160, max_len_answ=30, batch_size=256):
        super().__init__()
        self.module = module
        self.train_type = train_type #should be "easy","medium","hard"
        self.test_type = "interpolate"
        self.max_len_quest = max_len_quest
        self.max_len_answ = max_len_answ
        self.batch_size = batch_size
        self.dataset = MathematicsDataset(self.module, self.train_type) #change if needed
        self.vocab = self.dataset.vocab
    # def prepare_data(self):
        # tok, load, ecc...
        # MNIST(self.data_dir, train=True, download=True)
        # MNIST(self.data_dir, train=False, download=True)

    def setup(self, stage=None):

        # Assign train/val datasets for use in dataloaders
        if stage == "fit" or stage is None:
            self.train_ds = MathematicsDataset(self.module, self.train_type)

        # Assign test dataset for use in dataloader(s)
        if stage == "test" or stage is None:
            self.test_ds = MathematicsDataset(self.test_type, "algebra")

        if stage == "predict" or stage is None:
            self.predict_ds = MathematicsDataset(self.test_type, "algebra")

    def train_dataloader(self):
        return DataLoader(self.train_ds, batch_size=self.batch_size, shuffle = True, num_workers=2)

    def test_dataloader(self):
        return DataLoader(self.test_ds, batch_size=self.batch_size, drop_last=True)

    def predict_dataloader(self):
        return DataLoader(self.predict_ds, batch_size=self.batch_size)

In [299]:
dataset = PlMathematicsDataset("algebra", "complete",batch_size=512)

In [ ]:
# #Create dataloader
# train_loader = DataLoader(dataset, 32, shuffle=True)
# print(len(train_loader))

In [ ]:
# print("data", next(iter(train_loader)))
# print("questions_shape ", next(iter(train_loader))[0].shape)

# print("answers_shape ", next(iter(train_loader))[1].shape)

# Model

In [37]:
class EmbeddingLayer(nn.Module):
  def __init__(self,vocabulary_size,embedding_dim):
    super().__init__()
    self.E = nn.Embedding(vocabulary_size,embedding_dim)
  
  def forward(self,x):
    return self.E(x)

In [287]:
class LSTM(pl.LightningModule):
    def __init__(self,voc_size,hidden_size,emb_dim):
      super().__init__()
      self.save_hyperparameters()
      self.voc_size = voc_size
      self.hidden_size = hidden_size
      self.emb_dim = emb_dim
      self.embedding = EmbeddingLayer(voc_size,emb_dim)
      self.lstm1 = nn.LSTM(emb_dim, hidden_size, 1)
      self.lstm2 = nn.LSTM(emb_dim, hidden_size, 1)
      self.linear = nn.Linear(hidden_size, voc_size)
      self.metric = nn.CrossEntropyLoss(ignore_index=0)

    def training_step(self, batch, batch_idx):
        # training_step defines the train loop.
        quest, answ = batch
        output = self.forward(quest,answ,teacher_forcing_ratio=0.5)
        pred = output[:,1:,:].reshape(output.shape[0]*(output.shape[1]-1),output.shape[2])
        target = answ[:,1:].reshape(answ.shape[0]*(answ.shape[1]-1))
        loss = self.metric(pred,target)
        self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(),lr=6e-04, betas=(0.9, 0.995), eps=1e-09)
        return optimizer
    
    def accuracy(self,pred,targ,voc_size):
      c = 0
      for i in range(targ.shape[0]):
        index = (targ[i,:]==2).nonzero().item()
        val = True
        for j in range(index+1):
          if pred[i,j] != targ[i,j]:
            val = False
        if  val == True:
          c += 1
      return c/targ.shape[0]

    def test_step(self, batch, batch_idx):
        quest,answ = batch
        target_pad_mask = get_padding_mask(answ[:,1:],inverse = True)
        pred = self.forward(quest,answ)[:,1:,:]
        acc = self.accuracy(pred.argmax(-1),answ[:,1:],self.voc_size)
        self.log('test_acc', acc, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return acc

    def forward(self,quest,answ,teacher_forcing_ratio=0.0):

      device = 'cuda' if torch.cuda.is_available() else 'cpu'

      q = self.embedding(quest.T)
      outputs, (hidden_state, cell_state) = self.lstm1(q)
      
      output_sequence = torch.zeros(answ.shape[1], answ.shape[0], self.voc_size).to(device)

      a = answ.T
      char = a[0,:] 
      for t in range(answ.shape[1]):
        emb = self.embedding(char.unsqueeze(0))
        outputs, (hidden_state, cell_state) = self.lstm2(emb, (hidden_state, cell_state))
        linear_out = self.linear(outputs.squeeze(0))
        output_sequence[t] = linear_out

        forcing = random.random() < teacher_forcing_ratio
        if forcing == True:
          char = a[t,:]
        else:
          if t == 0:
            char = a[0,:]
          else:
            char = linear_out.argmax(-1)
  
      return output_sequence.transpose(0,1)


    def predict_step(self, batch, batch_idx):
        quest,answ = batch
        pad_mask = get_padding_mask(answ[:,1:])
        out = self.forward(quest,answ)
        res = out.argmax(dim=-1)[:,1:]
        res[pad_mask[:,0,:]] = 0
        return res

In [292]:
hidden_size = 2048
vocabulary_size = len(dataset.vocab)
emb_dim = 512
enable_checkpointing = True
resume_training = True
load_model = True
load_model_dir = "/content/drive/MyDrive/Deep learning/model_checkpoints/LSTM/last.ckpt" #change path to load a different checkpoint

In [293]:
model = LSTM(vocabulary_size,hidden_size,emb_dim)
if load_model == True:
  model = LSTM.load_from_checkpoint(load_model_dir)

In [294]:
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint

checkpoint_callback = ModelCheckpoint(dirpath="/content/drive/MyDrive/Deep learning/model_checkpoints/LSTM", save_last =True, save_on_train_epoch_end = True)
# define the logger object 
logger = TensorBoardLogger("/content/drive/MyDrive/Deep learning/tb_logs", name="LSTM",log_graph=True )

#build the correct callbacks
callbacks = [TQDMProgressBar(refresh_rate=20)]
ckpt_path = load_model_dir

#handle checkpointing
if resume_training == False:
  ckpt_path = None
if enable_checkpointing == True:
  callbacks = [checkpoint_callback, TQDMProgressBar(refresh_rate=20)]
  
#passing it to the trainer
gpu = 0
if torch.cuda.is_available() : 
  gpu = 1
trainer = pl.Trainer(enable_checkpointing=enable_checkpointing ,gpus=gpu, max_epochs=7, logger=logger, callbacks=callbacks)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [295]:
trainer.fit(ckpt_path=ckpt_path, model=model, datamodule=dataset)
print(checkpoint_callback.best_model_path)

INFO:lightning_fabric.accelerators.cuda:You are using a CUDA device ('NVIDIA A100-SXM4-40GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
/usr/local/lib/python3.8/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:612: UserWarning: Checkpoint directory /content/drive/.shortcut-targets-by-id/1lKplZjRr839VnzCbCDMjd--WHVhTwnbR/Deep learning/model_checkpoints/LSTM exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/drive/MyDrive/Deep learning/model_checkpoints/LSTM/last.ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightn

Training: 3907it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=7` reached.


/content/drive/.shortcut-targets-by-id/1lKplZjRr839VnzCbCDMjd--WHVhTwnbR/Deep learning/model_checkpoints/LSTM/epoch=6-step=27349.ckpt


In [300]:
trainer.test(model=model,datamodule=dataset)

INFO:lightning_fabric.accelerators.cuda:You are using a CUDA device ('NVIDIA A100-SXM4-40GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.8/dist-packages/pytorch_lightning/loggers/tensorboard.py:188: UserWarning: Could not log computational graph to TensorBoard: The `model.example_input_array` attribute is not set or `input_array` was not given.
  rank_zero_warn(
/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argu

Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     test_acc_epoch         0.48180508613586426
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_acc_epoch': 0.48180508613586426}]